# Apache MXNet Basic 실습
## Module - Neural network training and inference

학습 수행 순서학습 
* 데이터 입력
* 모델 파라메터 초기화
* 네트워크를 따라 Forward, Backward 수행
* Gradient들을 기반으로 weight 값 조정
* 모델 체크포인트

Luckily, MXNet modularizes commonly used code for training and inference in
the `module` (`mod` for short) package. `Module` provides both high-level and
intermediate-level interfaces for executing predefined networks. One can use
both interfaces interchangeably. We will show the usage of both interfaces in
this tutorial.


## Preliminary

이번 실습은 [UCI letter recognition](https://archive.ics.uci.edu/ml/datasets/letter+recognition) dataset을 대상으로 [Multilayer Perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP) 네트워크를 학습을 수행합니다.

### 1. 학습, 검증 데이터 준비

* Dataset을 다운로드하고 80:20 비율료 학습:테스트 데이터로 나눔
* training data iterator를 초기화해서 32개의 배치 샘플을 주도록 설정함
* 테스트 데이터를 위한 별도의 iterator 생성

> <font color='red'>문제</font>
1. 아래 코드는 학습:테스트 데이터가 5:5 비율로 되어 있습니다. 80:20 비율이 되도록 코드를 변경해보세요. (힌트: ntrain에 할당되는 값에 곱해지는 비율 변경)
2. 아래 코드는 batch 크기가 1로 되어 있습니다. 코드를 변경해서 batch 크기가 32가 되도록해보세요.

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)
import mxnet as mx
import numpy as np

fname = mx.test_utils.download('http://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data')
data = np.genfromtxt(fname, delimiter=',')[:,1:]
label = np.array([ord(l.split(',')[0])-ord('A') for l in open(fname, 'r')])

batch_size = 1
ntrain = int(data.shape[0]*0.5)
train_iter = mx.io.NDArrayIter(data[:ntrain, :], label[:ntrain], batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(data[ntrain:, :], label[ntrain:], batch_size)

### 2. 네트워크 정의

1) 첫번째 레이어(hidden layer)는 64개의 neuron을 갖는 fully-connected로 정의

2) 첫번째 레이어의 각 neuron에 relu activation 함수를 적용

3) 두번째 레이어(output layer)에는 26개의 neuron을 갖는 fully-connected로 정의

4) 두번째 레이어에는 multi classification을 위한 softmax 적용

In [ ]:
net = mx.sym.Variable('data')
net = mx.sym.FullyConnected(net, name='fc1', num_hidden=64)
net = mx.sym.Activation(net, name='relu1', act_type="relu")
net = mx.sym.FullyConnected(net, name='fc2', num_hidden=26)
net = mx.sym.SoftmaxOutput(net, name='softmax')
mx.viz.plot_network(net)

### 3. Module 생성

Module 생성은 `Module` 클래스를 사용하며, 다음과 같은 파라메터가 있음:

- `symbol`: Neural Network 정의
- `context`: 어떤 디바이스에서 수행할 것인지 (CPU, GPU, 여러 GPU들)
- `data_names` : 입력 데이터 변수 이름들
- `label_names` : 입력 label 변수 이름들

이 예제에서는 `data`라는 이름의 한개의 데이터를 사용하고, label은 `softwas_label`이라는 이름을 사용합니다. `softmax_label`은 `SoftmaxOutput` 연산 이름에서 자동으로 부여된 이름입니다.

In [ ]:
mod = mx.mod.Module(symbol=net,
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label'])

### 4-1 중간 레벨 인터페이스(Intermediate-level Interface)를 이용한 학습 수행

중간 레벨 인터페이스를 사용해서 모델을 학습 시키는 순서는 다음과 같습니다.

- `bind` : 메모리를 할당해서 연산 환경을 준비
- `init_params` : 파레메터 할당 및 초기화
- `init_optimizer` : optimizer 초기화 (기본 optimizer는 `sgd`)
- `metric.create` : 입력 metric 이름을 사용해서 평가 metric 생성
- `forward` : Forward 계산 수행
- `update_metric` : 마지막 forward 계산 결과를 바탕으로 평가 metric 업데이트 (Evaluates and accumulates)
- `backward` : Backward 계산 수행
- `update` : 위에서 설정된 optimizer와 마지막으로 계산된 forward-backward 배치 결과를 기반으로 파라메터들을 업데이트함

In [ ]:
# allocate memory given the input data and label shapes
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
# initialize parameters by uniform random numbers
mod.init_params(initializer=mx.init.Uniform(scale=.1))
# use SGD with learning rate 0.1 to train
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
# use accuracy as the metric
metric = mx.metric.create('acc')
# train 5 epochs, i.e. going over the data iter one pass
for epoch in range(8):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        mod.forward(batch, is_train=True)       # compute predictions
        mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
        mod.backward()                          # compute gradients
        mod.update()                            # update parameters
    print('Epoch %d, Training %s' % (epoch, metric.get()))

To learn more about these APIs, visit [Module API](http://mxnet.io/api/python/module.html).

### 4-2 상위 레벨 인터패이스 (High-level Interface)를 이용한 학습, 예측

### 학습

[fit API](http://mxnet.io/api/python/module.html#mxnet.module.BaseModule.fit) 하나를 호출하면, 위 과정들이 자동으로 수행됩니다.

In [ ]:
# reset train_iter to the beginning
train_iter.reset()

# create a module
mod = mx.mod.Module(symbol=net,
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label'])

# fit the module
mod.fit(train_iter,
        eval_data=val_iter,
        optimizer='sgd',
        optimizer_params={'learning_rate':0.1},
        eval_metric='acc',
        num_epoch=8)

By default, `fit` function has `eval_metric` set to `accuracy`, `optimizer` to `sgd`
and optimizer_params to `(('learning_rate', 0.01),)`.

### 예측 및 평가 (Predict and Evaluate)

`predict()`를 사용해서 예측을 수행합니다. 이 함수는 모든 예측 결과를 수집해서 반환합니다.

In [ ]:
y = mod.predict(val_iter)
assert y.shape == (4000, 26)

만약 예측 결과는 필요없고 테스트 데이터를 사용해서 학습된 모델을 평가만 필요하면 `score()` 함수를 이용할 수 있습니다. 이 함수는 input validation dataset을 대상으로 예측값을 계산하고, 정의된 input metric을 사용해서 모델의 성능을 평가합니다.

In [ ]:
score = mod.score(val_iter, ['acc'])
print("Accuracy score is %f" % (score[0][1]))

평가를 위한 다른 metric으로는 `top_k_acc`(top-k-accuracy),
`F1`, `RMSE`, `MSE`, `MAE`, `ce`(CrossEntropy) 등이 있습니다. 자세한 내용은 [Evaluation metric](http://mxnet.io/api/python/metric.html) 참고하세요.

> <font color='red'>문제</font>
학습 반복 획수(epoch), 학습 속도(learning_rate), 최적화 파라메터를 변경하면서 score 값이 어떻게 바뀌는지 살펴보고, 최적의 score를 얻는 설정을 찾아보세요.

### 5. 학습 모델 저장

학습 epoch 마다 checkpoint callback을 사용해서 module 파라메터들을 저장할 수 있습니다.

In [ ]:
# construct a callback function to save checkpoints
model_prefix = 'mx_mlp'
checkpoint = mx.callback.do_checkpoint(model_prefix)

mod = mx.mod.Module(symbol=net)
mod.fit(train_iter, num_epoch=5, epoch_end_callback=checkpoint)

> <font color='red'>문제</font>
저장된 symbol 파일 내용을 확인해보세요.

> 1) !ls (Windows는 dir)로 파일 이름 확인 (mx_mlp-symbol.json)
>
> 2) cat (Windows는 type) 으로 파일 내용 확인

In [ ]:
!ls

In [ ]:
!cat ___symbol 파일 이름____

### 6. 저당된 학습 모델  읽기

저장된 module parameter를 읽어서 사용하기 위해서 `load_checkpoint` 함수를 사용해서 저장된 module parameter를 읽을 수 있습니다. 이 함수는 Symbol과 그와 연관된 파라메터를 읽어 들입니다. 이 파라메터는 module에 설정해서 사용할 수 있습니다.

In [ ]:
sym, arg_params, aux_params = mx.model.load_checkpoint(model_prefix, 3)
assert sym.tojson() == net.tojson()

# assign the loaded parameters to the module
mod.set_params(arg_params, aux_params)

만약, 저장된 체크포인트에 이어서 학습을 계속 수행하고 싶으면 `set_params()`를 사용하는 대신 `fit()` 함수를 사용합니다. 이렇게 하면, 파라메터들을 초기화하지 않고 학습을 재시작하고, `begin_epoch` 변수를 사용하면 마지막 저장된 epoch부터 재시작한다는 것을 알릴 수 있습니다. 만약 저장된 파라메터가 epoch 2에서 였다면, begin_epoch는 3으로 설정합니다.

In [ ]:
mod = mx.mod.Module(symbol=sym)
mod.fit(train_iter,
        num_epoch=8,
        arg_params=arg_params,
        aux_params=aux_params,
        begin_epoch=3)


<!-- INSERT SOURCE DOWNLOAD BUTTONS -->

